In [1]:
import numpy as np
import pandas as pd
import os
import re

import src.SimData  as sim_data

In [2]:

###* Data LOKI Jorge 

def get_species(col):
    match = re.match(r'\[(.*?)\]', col)
    if match:
        return match.group(1).split('(')[0]
    return col


folder_path = "Experimental_data_CO/ResultadosLoKIJorge"
file_name_list = os.listdir(folder_path)

data_files_path = [os.path.join(folder_path, ele) for ele in file_name_list if ele.endswith(".txt")]

print(data_files_path)

pd_list = []
for idx, file_path in enumerate(data_files_path):
    
    df = pd.read_csv(file_path, delimiter=" ")
    
    species_groups = {}
    for col in df.columns:
        species = get_species(col)
        if species not in species_groups:
            species_groups[species] = []
        species_groups[species].append(col)
    
    species_df = pd.DataFrame()
    for species, cols in species_groups.items():
        species_df[species] = df[cols].sum(axis=1)
        
    pd_list.append(species_df)



result_pd = pd.concat(pd_list, ignore_index=True)

print(result_pd.head())
# print(result_pd['fracCO2'].tolist())
# print(result_pd['fracO2'].tolist())
# print(result_pd.head())

schema_LoKI = {
    "p(Torr)":          "pressure",
    "I(mA)":            "current",
    "fracCO2":          "frac_CO2",
    "GasTemp(K)":       "Tgas",
    "Twall":            "Tw",
    "NearWallTemp(K)":  "Tnw",
    "O":                "O",
    "O2":               "O2",
    "O3":               "O3",
    "C":                "C",
    "CO":               "CO",
    "CO2":              "CO2"
}

pd_result_LOKI = pd.DataFrame()

for old, new in schema_LoKI.items():
    if old in result_pd.columns:
        pd_result_LOKI[new] = result_pd[old]


print(pd_result_LOKI.head())
print(pd_result_LOKI.shape)

pd_result_LOKI.to_csv("result_LoKI.csv", index=False)


['Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_0.txt', 'Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_025.txt', 'Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_05.txt', 'Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_0_lowT.txt', 'Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_1_lowT.txt', 'Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_05_lowT.txt', 'Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_075_lowT.txt', 'Experimental_data_CO/ResultadosLoKIJorge/summaryResults_fracCO2_075.txt']
   p(Torr)  I(mA)   Twall  Flow(sccm)  fracCO2  fracO2  lossFreq(s-1)  \
0      1.0     40  323.15       7.791      0.0     1.0      28.031203   
1      1.5     40  323.15       7.791      0.0     1.0      30.964461   
2      2.0     40  323.15       7.791      0.0     1.0      34.499889   
3      3.0     40  323.15       7.791      0.0     1.0      40.208772   
4      5.0     40  323

In [3]:


###* Experimental Data CO 2

schema = {
    "Pressure":     "pressure",
    "CurrentName":  "current",
    "fracCO2ini_relO2": "frac_CO2",
    "WallTemp":     "Tw",
    "T near wall":  "Tnw", 
    "Tgas":         "Tgas",
    "O mean":       "O",
    "CO mean":      "CO",
    "MeanLossProb": "gamma_exp"
}

file_path = "Experimental_data_CO/Data_CO2_TD.xlsx"

pd_exp = pd.read_excel(file_path)
pd_result_exp = pd.DataFrame()

for old, new in schema.items():
    if old in pd_exp.columns:
        pd_result_exp[new] = pd_exp[old]


pd_result_exp["Tw"] = pd_result_exp["Tw"] + 273.15 

print(pd_result_exp.head())
print(pd_result_exp.shape)


pd_result_exp.to_csv("result_exp.csv", index=False)



   pressure  current  frac_CO2      Tw        Tgas  gamma_exp
0       0.4       40       0.0  323.15  360.560032   0.001819
1       0.6       40       0.0  323.15  372.733684   0.000995
2       0.8       40       0.0  323.15  384.630566   0.000841
3       1.0       40       0.0  323.15  396.252108   0.000775
4       1.5       40       0.0  323.15  424.110735   0.000827
(332, 6)


In [4]:

###* Merge Both DataFrames

common_cols = ["pressure", "current", "Tw", "frac_CO2"]
merged_df = pd.merge(pd_result_LOKI, pd_result_exp, on=common_cols, how='inner')


merged_df['Tw'] = merged_df['Tw'] - 273.15
merged_df['Tgas'] = 0.5*(merged_df['Tgas_x'] + merged_df['Tgas_y'])

merged_df['O'] = merged_df['O'] * 1e-6
merged_df['O2'] = merged_df['O2'] * 1e-6
merged_df['O3'] = merged_df['O3'] * 1e-6

merged_df['C'] = merged_df['C'] * 1e-6
merged_df['CO'] = merged_df['CO'] * 1e-6
merged_df['CO2'] = merged_df['CO2'] * 1e-6


merged_df.to_excel("Experimental_data_CO/Experimental_data_CO.xlsx", index=False)


print(merged_df.head())
print(merged_df.shape)
print(merged_df.columns)



   pressure  current  frac_CO2      Tgas_x    Tw         Tnw             O  \
0       1.0       40       0.0  393.444192  50.0  332.243200  8.157239e+15   
1       1.0       40       0.0  393.444192  50.0  332.243200  8.157239e+15   
2       1.0       40       0.0  394.414880  50.0  332.346773  7.734088e+15   
3       1.0       40       0.0  394.414880  50.0  332.346773  7.734088e+15   
4       1.5       40       0.0  413.891596  50.0  335.174979  9.699256e+15   

             O2            O3    C   CO  CO2      Tgas_y  gamma_exp  \
0  1.638529e+16  5.460660e+11  0.0  0.0  0.0  396.252108   0.000775   
1  1.638529e+16  5.460660e+11  0.0  0.0  0.0  396.252108   0.000843   
2  1.674804e+16  5.432798e+11  0.0  0.0  0.0  396.252108   0.000775   
3  1.674804e+16  5.432798e+11  0.0  0.0  0.0  396.252108   0.000843   
4  2.529634e+16  8.603773e+11  0.0  0.0  0.0  424.110735   0.000827   

         Tgas  
0  394.848150  
1  394.848150  
2  395.333494  
3  395.333494  
4  419.001166  
(94, 15)

In [5]:

### store in buffer data in hdf5 style

schema_buffer = {
    "pressure":     "pressure",
    "current":      "current",
    "frac_CO2":     "frac_CO2",
    "Tgas":         "Tgas",
    "Tnw":          "Tnw", 
    "Tw":           "Tw",
    "O":            "O",
    "O2":           "O2",
    "O3":           "O3",
    "C":            "C",
    "CO":           "CO",
    "CO2":          "CO2",
    "gamma_exp":    "gamma_exp"
}


output_file = "Experimental_data_CO_Jorge.hdf5"
file_path = ["Experimental_data_CO/Experimental_data_CO.xlsx"]


data_loader = sim_data.DataLoader(schema_buffer, output_file=output_file, files_path=file_path)
data_loader.load_data(force_update=True)

